### Import libraries and create DataFrame from csv 

In [33]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import Ridge
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_squared_error

# Function to read, preprocess, and aggregate full season data
def read_data():
    # Read the full season data
    data = pd.read_excel("WR_Data_2024.xlsx")

    # Preprocess full season data, drop irrelevant columns and convert data types
    data.drop(['FL', 'ROST'], axis=1, inplace=True)
    convert = data.select_dtypes('object').columns.difference(['Player'])
    data[convert] = data[convert].apply(lambda x: pd.to_numeric(x.str.replace(',', ''), errors='coerce')).fillna(0)
    data['Rank'] = data['Rank'].astype('Int64')

    return data

# Load the full season data
df = read_data()

# Recalculate 'Rank' based on 'FPTS' in descending order
df['Rank'] = df['FPTS'].rank(ascending=False, method='min').astype('int')

# Sort the DataFrame by 'Rank'
df = df.sort_values(by='Rank')
df.head(20)

,Rank,Player,REC,TGT,YDS,Y/R,LG,20+,TD,ATT,YDS.1,TD.1,G,FPTS,FPTS/G
0,1,Chris Godwin (TB),15,16,200,13.3,41,2,2,0,0,0,2,47.0,23.5
1,2,Nico Collins (HOU),14,18,252,18.0,55,3,1,0,0,0,2,45.2,22.6
2,3,Malik Nabers (NYG),15,25,193,12.9,28,5,1,0,0,0,2,40.3,20.2
3,4,Jayden Reed (GB),6,8,147,24.5,70,3,1,3,70,1,2,39.7,19.9
4,4,Cooper Kupp (LAR),18,27,147,8.2,24,3,1,2,10,0,2,39.7,19.9
5,6,Justin Jefferson (MIN),8,13,192,24.0,97,2,2,0,0,0,2,39.2,19.6
6,7,Jameson Williams (DET),10,20,200,20.0,52,4,1,2,28,0,2,38.8,19.4
7,8,Alec Pierce (IND),8,10,181,22.6,60,2,2,0,0,0,2,38.1,19.1
8,9,Rashid Shaheed (NO),7,9,169,24.1,70,2,2,3,13,0,2,37.2,18.6
9,10,Davante Adams (LV),14,18,169,12.1,30,4,1,0,0,0,2,36.9,18.5


### Convert relevant stats to a per-game basis and create the final stats for analysis

In [34]:
# Define columns for per-game calculation
per_game = list(df.columns[2:])

# Define columns to exclude from the per-game calculation
exclude_per_game = ['Y/R', 'LG', 'G', 'FPTS', 'FPTS/G']

# Create columns and convert stats to a per-game basis for the defined columns, not those in the exclude list
for col in per_game:
    if col not in exclude_per_game:
        df[col + '/game'] = (df[col] / df['G']).round(1)

# Define final columns to be used for analysis, combining excluded and new per-game columns
final_columns = exclude_per_game + [col + '/game' for col in per_game if col not in exclude_per_game]

# Display final columns with 'Rank' and 'Player'
df[['Rank', 'Player'] + final_columns].head(10)

,Rank,Player,Y/R,LG,G,FPTS,FPTS/G,REC/game,TGT/game,YDS/game,20+/game,TD/game,ATT/game,YDS.1/game,TD.1/game
0,1,Chris Godwin (TB),13.3,41,2,47.0,23.5,7.5,8.0,100.0,1.0,1.0,0.0,0.0,0.0
1,2,Nico Collins (HOU),18.0,55,2,45.2,22.6,7.0,9.0,126.0,1.5,0.5,0.0,0.0,0.0
2,3,Malik Nabers (NYG),12.9,28,2,40.3,20.2,7.5,12.5,96.5,2.5,0.5,0.0,0.0,0.0
3,4,Jayden Reed (GB),24.5,70,2,39.7,19.9,3.0,4.0,73.5,1.5,0.5,1.5,35.0,0.5
4,4,Cooper Kupp (LAR),8.2,24,2,39.7,19.9,9.0,13.5,73.5,1.5,0.5,1.0,5.0,0.0
5,6,Justin Jefferson (MIN),24.0,97,2,39.2,19.6,4.0,6.5,96.0,1.0,1.0,0.0,0.0,0.0
6,7,Jameson Williams (DET),20.0,52,2,38.8,19.4,5.0,10.0,100.0,2.0,0.5,1.0,14.0,0.0
7,8,Alec Pierce (IND),22.6,60,2,38.1,19.1,4.0,5.0,90.5,1.0,1.0,0.0,0.0,0.0
8,9,Rashid Shaheed (NO),24.1,70,2,37.2,18.6,3.5,4.5,84.5,1.0,1.0,1.5,6.5,0.0
9,10,Davante Adams (LV),12.1,30,2,36.9,18.5,7.0,9.0,84.5,2.0,0.5,0.0,0.0,0.0


### Calculate the correlations for the final stats across different conditions

In [35]:
# Define columns and columns to exclude for correlation calculation, excluding FPTS and FPTS/G
exclude_corr = ['FPTS/G', 'FPTS', 'G']
corr_columns = [col for col in final_columns if col not in exclude_corr]

# Define a function to calculate correlations
def compute_correlations(dataframe, corr_columns):
    return dataframe[corr_columns].corrwith(dataframe['FPTS/G'])

# Compute correlations for various conditions
corr_all = compute_correlations(df, corr_columns)
corr_nonzero = compute_correlations(df[df['FPTS/G'] > 0], corr_columns)
corr_top50 = compute_correlations(df[df['Rank'] <= 50], corr_columns)
corr_top25 = compute_correlations(df[df['Rank'] <= 25], corr_columns)

# Compile all correlations into a DataFrame for comparison
df_corr = pd.DataFrame({
    'All Players': corr_all,
    'FPTS > 0': corr_nonzero,
    'Top 50 Players': corr_top50,
    'Top 25 Players': corr_top25
})

# Calculate the average correlation across the four conditions, adding 'Average' column to DataFrame
df_corr['Correlation'] = df_corr.mean(axis=1)

# Display the correlation DataFrame
df_corr.round(2)

,All Players,FPTS > 0,Top 50 Players,Top 25 Players,Correlation
Y/R,0.72,0.41,0.32,0.30,0.44
LG,0.85,0.71,0.45,0.31,0.58
REC/game,0.90,0.85,0.57,0.32,0.66
TGT/game,0.85,0.79,0.53,0.12,0.57
YDS/game,0.96,0.95,0.83,0.82,0.89
20+/game,0.79,0.76,0.53,0.53,0.65
TD/game,0.74,0.72,0.53,0.10,0.52
ATT/game,0.28,0.22,0.20,0.11,0.20
YDS.1/game,0.33,0.29,0.27,0.17,0.26
TD.1/game,0.27,0.27,0.18,0.09,0.20


### Assign the weights for the final stats

In [36]:
# Calculate R^2 for the 'Average' correlation, adding 'R^2' column to DataFrame
df_corr['R^2'] = df_corr['Correlation'] ** 2

# Select stats with R^2 above a threshold for higher weight
high_weight_threshold = 0.5
specific_stats = df_corr[df_corr['R^2'] > high_weight_threshold].index.tolist()

# Select stats with R^2 below a threshold to exclude from final score
exclude_threshold = 0.1
exclude_stats = df_corr[df_corr['R^2'] < exclude_threshold].index.tolist()

# Define the calculation to assign weights
def weight_calc(row, specific_stats):
    if row.name in specific_stats:
        return 1 + row['R^2'] * 2 # Assign higher weight to specific stats
    else:
        return 1 + row['R^2'] # Assign weight to all other stats
    
# Assign weights based on the given criteria, adding 'Weight' column to DataFrame
df_corr['Weight'] = df_corr.apply(weight_calc, specific_stats=specific_stats, axis=1)

# Normalize weights
df_corr['Weight'] = df_corr['Weight'] / df_corr['Weight'].sum()

# Display the new columns in the DataFrame
df_corr[['Correlation', 'R^2', 'Weight']].round(2)

,Correlation,R^2,Weight
Y/R,0.44,0.19,0.09
LG,0.58,0.33,0.10
REC/game,0.66,0.43,0.10
TGT/game,0.57,0.33,0.10
YDS/game,0.89,0.79,0.19
20+/game,0.65,0.42,0.10
TD/game,0.52,0.27,0.09
ATT/game,0.20,0.04,0.08
YDS.1/game,0.26,0.07,0.08
TD.1/game,0.20,0.04,0.08


### Multiply the assigned weights to the final stats and calculate the score

In [37]:
# Multiply each relevant column by its corresponding weight
for col in corr_columns:
    if col not in exclude_stats:
        weight = df_corr.loc[col, 'Weight']
        df[col + '_weighted'] = (df[col] * weight).round(1)

# Extract the weighted columns
weight_columns = [col + '_weighted' for col in corr_columns if col not in exclude_stats]

# Display the new weighted stats columns
df_weight = df[['Rank', 'Player', 'FPTS/G'] + weight_columns]
df_weight.sort_values(by=['Rank'], ascending=True).head(10)

# Define select columns to be used for the average weighted score
avg = weight_columns + (['FPTS/G'] * 2)  # Giving higher weight to FPTS/G

# Calculate the average weighted score for the select columns
df['Score'] = df[avg].mean(axis=1).round(2)

# Normalize the scores to be out of 10
scaler = MinMaxScaler(feature_range=(0, 10))
df['Score'] = scaler.fit_transform(df[['Score']])

df_weight.head(10)

,Rank,Player,FPTS/G,Y/R_weighted,LG_weighted,REC/game_weighted,TGT/game_weighted,YDS/game_weighted,20+/game_weighted,TD/game_weighted
0,1,Chris Godwin (TB),23.5,1.2,4.0,0.8,0.8,18.8,0.1,0.1
1,2,Nico Collins (HOU),22.6,1.6,5.4,0.7,0.9,23.7,0.2,0.0
2,3,Malik Nabers (NYG),20.2,1.1,2.7,0.8,1.2,18.2,0.3,0.0
3,4,Jayden Reed (GB),19.9,2.1,6.8,0.3,0.4,13.8,0.2,0.0
4,4,Cooper Kupp (LAR),19.9,0.7,2.3,0.9,1.3,13.8,0.2,0.0
5,6,Justin Jefferson (MIN),19.6,2.1,9.4,0.4,0.6,18.1,0.1,0.1
6,7,Jameson Williams (DET),19.4,1.7,5.1,0.5,1.0,18.8,0.2,0.0
7,8,Alec Pierce (IND),19.1,2.0,5.8,0.4,0.5,17.0,0.1,0.1
8,9,Rashid Shaheed (NO),18.6,2.1,6.8,0.4,0.4,15.9,0.1,0.1
9,10,Davante Adams (LV),18.5,1.1,2.9,0.7,0.9,15.9,0.2,0.0


### Model training

In [38]:
# Prepare features and target for model training
X = df[weight_columns]
y = df['Score']

# Handle missing values by imputing with mean
imputer = SimpleImputer(strategy='mean')
X = imputer.fit_transform(X)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test, train_idx, test_idx = train_test_split(X, y, df.index, test_size=0.2, random_state=42)

# Initialize Ridge regression model
ridge_model = Ridge()

# Cross-validation to evaluate the model
cv_scores = cross_val_score(ridge_model, X_train, y_train, cv=5, scoring='neg_mean_squared_error')
print(f'Cross-Validation MSE: {-cv_scores.mean()}')

# Train the model
ridge_model.fit(X_train, y_train)

# Predict and evaluate on the test set
y_pred = ridge_model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print(f'Test Set MSE: {mse}')

# Incorporate Random Forest as an ensemble method
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)
rf_pred = rf_model.predict(X_test)
rf_mse = mean_squared_error(y_test, rf_pred)
print(f'Random Forest Test Set MSE: {rf_mse}')

# Average predictions from both models for final score
final_pred = (y_pred + rf_pred) / 2

# Create a DataFrame for the test set predictions
test_results = pd.DataFrame({'Final_Score': final_pred}, index=test_idx)

# Merge the test set predictions back into the original DataFrame
df = df.merge(test_results, how='left', left_index=True, right_index=True)

# Fill NaN values in 'Final_Score' column with the original 'Score' to handle missing indices
df['Final_Score'].fillna(df['Score'], inplace=True)

# Normalize final scores to be out of 10
df['Final_Score'] = scaler.fit_transform(df[['Final_Score']]).round(2)

# Rank the final scores
df['Final Rank'] = df['Final_Score'].rank(method='first', ascending=False).astype(int)

# Calculate the variance in ranking both ranks
df['Variance'] = df['Rank'] - df['Final Rank']

Cross-Validation MSE: 0.49609636298151233
Test Set MSE: 0.5607912171697458
Random Forest Test Set MSE: 0.06411506157763755


/tmp/ipykernel_3897/591390400.py:44: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Final_Score'].fillna(df['Score'], inplace=True)


### Display final results and export to Excel

In [39]:
# List of columns to be excluded
final_columns_exclude = ['Y/R', 'LG', 'ATT/game', 'YDS.1/game', 'TD.1/game']

# Exclude the specified columns from final_columns
final_columns = [col for col in final_columns if col not in final_columns_exclude]

# Create final analysis columns
analysis = df[['Rank', 'Final Rank', 'Player', 'Final_Score', 'Variance'] + final_columns]
analysis.set_index('Rank', inplace=True)
analysis = analysis.sort_values(by='Final Rank', ascending=True)

# Export to Excel
analysis.to_excel("WR_Analysis.xlsx", index=False)

# Display the top 30 rows
analysis.head(30)

,Final Rank,Player,Final_Score,Variance,G,FPTS,FPTS/G,REC/game,TGT/game,YDS/game,20+/game,TD/game
Rank,,,,,,,,,,,,
32,1,A.J. Brown (PHI),10.00,31,1,22.9,22.9,5.0,10.0,119.0,2.0,1.0
2,2,Nico Collins (HOU),9.89,0,2,45.2,22.6,7.0,9.0,126.0,1.5,0.5
6,3,Justin Jefferson (MIN),9.60,3,2,39.2,19.6,4.0,6.5,96.0,1.0,1.0
1,4,Chris Godwin (TB),9.30,-3,2,47.0,23.5,7.5,8.0,100.0,1.0,1.0
7,5,Jameson Williams (DET),8.47,2,2,38.8,19.4,5.0,10.0,100.0,2.0,0.5
3,6,Malik Nabers (NYG),8.31,-3,2,40.3,20.2,7.5,12.5,96.5,2.5,0.5
9,7,Rashid Shaheed (NO),8.10,2,2,37.2,18.6,3.5,4.5,84.5,1.0,1.0
8,8,Alec Pierce (IND),7.93,0,2,38.1,19.1,4.0,5.0,90.5,1.0,1.0
13,9,Rashee Rice (KC),7.67,4,2,35.8,17.9,6.0,7.5,89.0,1.0,0.5
